In [ ]:
#This is a variant of "taketwo-dsmvp-v1" for the Jigsaw data.

In [ ]:
#It is also a variant of the version that was pushed to the main branch of TakeTwo repo in Feb 21. 

In [ ]:
#This is a version of "v1" demo notebook.

In [ ]:
#CODE DEPENDENCY: sklearn, numpy, pandas, random, string. It uses from sklearn NB (Naive Bayes) and SGD (Support Vector Machine)

In [ ]:
#DATA DEPENDENCY: In this version the "Jigsaw unintended biased in toxicity" data is used for demo purpose. 

In [ ]:
#See https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data?select=train.csv

In [ ]:
#for detailed description of the data and instructions to download them. 

In [ ]:
#As of now we are using only the "train.csv" portion of the data, which needs to be in the calling directory. 

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import random

In [ ]:
import string

In [ ]:
#Fetch the Jigsaw data and store them in appropriate arrays

In [ ]:
jigsawTrain = pd.read_csv('train.csv')

In [ ]:
jigsawTrain.shape

In [ ]:
allTrain = pd.DataFrame(jigsawTrain, columns = ['id', 'comment_text', 'severe_toxicity'])

In [ ]:
allTrain.shape

In [ ]:
allTrain1M = allTrain.head(1000000)

In [ ]:
allTrain1M.shape

In [ ]:
#allTrain = allTrain1M

In [ ]:
blackData = jigsawTrain[(jigsawTrain.black == 1.0)]

In [ ]:
allTrain.shape

In [ ]:
blackTrain = pd.DataFrame(blackData, columns = ['id', 'comment_text', 'severe_toxicity', 'black'])

In [ ]:
blackTrain.shape

In [ ]:
blackTrainId = blackTrain['id']

In [ ]:
blackTrainText = blackTrain['comment_text']

In [ ]:
blackTrainSevereToxicity = blackTrain.loc[:,'severe_toxicity']

In [ ]:
blackTrainText.head(1)

In [ ]:
allTrainId = allTrain['id']

In [ ]:
allTrainText = allTrain['comment_text']

In [ ]:
allTrainSevereToxicity = allTrain.loc[:,'severe_toxicity']

In [ ]:
allTrainSevereToxicity.head()

In [ ]:
blackTrain.head(5)

In [ ]:
allTrain.head(5)

In [ ]:
blackTrainId.iloc[0]

In [ ]:
blackTrainText.iloc[0]

In [ ]:
blackTrainSevereToxicity.iloc[0]

In [ ]:
len(blackTrain)

In [ ]:
categories = ["non-toxic", "toxic"]

In [ ]:
from collections import namedtuple
data_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [ ]:
def flipCoin(bias):
    f = random.random()
    return True if f<bias else False

In [ ]:
for i in range(0,10):
    print(flipCoin(0.25))

In [ ]:
for x in range(len(blackTrain)):
    if (blackTrainSevereToxicity.iloc[x] > 0):
        data_set.target.append(1)
        data_set.urls.append(blackTrainId.iloc[x])
        data_set.data.append(blackTrainText.iloc[x])
    elif flipCoin(0.25):
        data_set.target.append(0)
        data_set.urls.append(blackTrainId.iloc[x])
        data_set.data.append(blackTrainText.iloc[x])

In [ ]:
from collections import namedtuple
all_data_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [ ]:
from collections import namedtuple
all_train_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [ ]:
from collections import namedtuple
all_test_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [ ]:
downSampling = 0.06

In [ ]:
for x in range(len(allTrain)):
    if (allTrainSevereToxicity.iloc[x] > 0):
        all_data_set.target.append(1)
        all_data_set.urls.append(allTrainId.iloc[x])
        all_data_set.data.append(allTrainText.iloc[x])
    elif flipCoin(downSampling):
        all_data_set.target.append(0)
        all_data_set.urls.append(allTrainId.iloc[x])
        all_data_set.data.append(allTrainText.iloc[x])

In [ ]:
#Here is a version in which training and test data are separately generated from all data

In [ ]:
trainRatio = 0.9

In [ ]:
for x in range(len(allTrain)):
    if (allTrainSevereToxicity.iloc[x] > 0):
        if flipCoin(trainRatio): 
            all_train_set.target.append(1)
            all_train_set.urls.append(allTrainId.iloc[x])
            all_train_set.data.append(allTrainText.iloc[x])
        else: 
            all_test_set.target.append(1)
            all_test_set.urls.append(allTrainId.iloc[x])
            all_test_set.data.append(allTrainText.iloc[x])
    elif flipCoin(downSampling):
        if flipCoin(trainRatio): 
            all_train_set.target.append(0)
            all_train_set.urls.append(allTrainId.iloc[x])
            all_train_set.data.append(allTrainText.iloc[x])
        else: 
            all_test_set.target.append(0)
            all_test_set.urls.append(allTrainId.iloc[x])
            all_test_set.data.append(allTrainText.iloc[x])

In [ ]:
#either leave data_set as is (generated from blackData) or set it to all_data_set (generated from all the data)

In [ ]:
data_set = all_test_set

In [ ]:
len(data_set.target)

In [ ]:
 data_set.target[0:10]

In [ ]:
sum(data_set.target)

In [ ]:
data_set.urls[0:10]

In [ ]:
data_set.data[0:2]

In [ ]:
marker_train = all_train_set

In [ ]:
#Here a separate test data set should be used.

In [ ]:
marker_test = all_test_set

In [ ]:
for t in marker_train.target[:10]:
...     print(marker_train.target_names[t])

In [ ]:
#marker_train.data is an array of strings (containing as many "texts" as there are coming from Marker.)

In [ ]:
#marker_train.target is an array of integers (containing the class indices for the above texts' class labels)

In [ ]:
#marker_train_target_names is an array of class label names indexed by the array indices.

In [ ]:
len(marker_train.target)

In [ ]:
sum(marker_train.target)

In [ ]:
len(marker_test.data)

In [ ]:
for t in marker_train.target[:10]:
...     print(marker_train.target_names[t])

In [ ]:
for t in marker_test.target[:10]:
...     print(marker_train.target_names[t])

In [ ]:
#Classes for model representations packaging the components in the modeling pipeline.
#For SGDClassifier and NaiveBayes.
#We may want to extend it to utilize "Pipeline()" as in "ModelRepSGD0", which is not in use at the moment.

In [ ]:
class ModelRepSGD:
    def __init__(self, count_vect, X_train_counts, tf_transformer, text_clf):
        self.count_vect = CountVectorizer()
        self.X_train_counts = self.count_vect.fit_transform(marker_train.data)
        self.tf_transformer = TfidfTransformer(use_idf=False).fit(self.X_train_counts)
        self.text_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                  alpha=1e-3, random_state=42,
                                  max_iter=5, tol=None)),
        ])

In [ ]:
class ModelRepNB:
    def __init__(self, count_vect, X_train_counts, tf_transformer, clf):
        self.count_vect = CountVectorizer()
        self.X_train_counts = self.count_vect.fit_transform(marker_train.data)
        self.tf_transformer = TfidfTransformer(use_idf=False).fit(self.X_train_counts)
        self.clf = MultinomialNB()

In [ ]:
#Functions for model training - for SGDClassifier and NaiveBayes (SGD one is untested)

In [ ]:
def model_train_SGD(marker_train): 
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(marker_train.data)
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
    X_train_tf = tf_transformer.transform(X_train_counts)
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', SGDClassifier(loss='hinge', penalty='l2',
                              alpha=1e-3, random_state=42,
                              max_iter=5, tol=None)),
    ])
    text_clf.fit(marker_train.data, marker_train.target)
    trained_model = ModelRepSGD(count_vect, X_train_counts, tf_transformer, text_clf)
    trained_model.text_clf = text_clf
    return trained_model

In [ ]:
def model_train_NB(marker_train): 
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(marker_train.data)
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
    X_train_tf = tf_transformer.transform(X_train_counts)
    clf = MultinomialNB().fit(X_train_tf, marker_train.target)
    trained_model = ModelRepNB(count_vect, X_train_counts, tf_transformer, clf)
    trained_model.clf = clf
    return trained_model

In [ ]:
#Function for model testing (common for SGDClassifier and NB)

In [ ]:
def model_test(flagger_new_test, trained_model):
    count_vect = trained_model.count_vect
    tf_transformer = trained_model.tf_transformer
    clf = trained_model.clf
    X_new_counts = count_vect.transform(flagger_new_test)
    X_new_tfidf = tf_transformer.transform(X_new_counts)
    predicted = clf.predict(X_new_tfidf)
    return predicted

In [ ]:
#Now testing model_train and model_test

In [ ]:
trained_model_NB = model_train_NB(marker_train)

In [ ]:
trained_model_SGD = model_train_SGD(marker_train)

In [ ]:
#Inspecting the components of the trained model

In [ ]:
type(trained_model_NB)

In [ ]:
trained_model_NB.count_vect

In [ ]:
trained_model_NB.tf_transformer

In [ ]:
trained_model_NB.clf

In [ ]:
trained_model_SGD.count_vect

In [ ]:
trained_model_SGD.tf_transformer

In [ ]:
trained_model_SGD.text_clf

In [ ]:
trained_model_SGD.count_vect

In [ ]:
trained_model_SGD.tf_transformer

In [ ]:
#Calling model tester/predictor on a single new text from Flagger

In [ ]:
#flagger_new_text = fetch_20newsgroups(subset='test',
#                                      categories=categories, shuffle=True, random_state=42)

In [ ]:
flagger_new_text = ['haha you guys are a bunch of losers.']

In [ ]:
predicted_NB = model_test(flagger_new_text, trained_model_NB)

In [ ]:
predicted_SGD = trained_model_SGD.text_clf.predict(flagger_new_text)

In [ ]:
predicted_NB

In [ ]:
predicted_SGD

In [ ]:
#Now evaluating NB and SGD models on a test data set

In [ ]:
predicted_test_NB = model_test(marker_test.data, trained_model_NB)

In [ ]:
predicted_test_SGD = trained_model_CREDENT.text_clf.predict(marker_test.data)

In [ ]:
predicted_test_SGD.shape

In [ ]:
sum(predicted_test_SGD)

In [ ]:
sum(predicted_test_NB)

In [ ]:
predicted_test_SGD[0:20]

In [ ]:
np.mean(predicted_test_NB == marker_test.target)

In [ ]:
np.mean(predicted_test_SGD == marker_test.target)

In [ ]:
!pip install -U ibm_watson_machine_learning
!pip install -U ibm-cos-sdk-core


In [ ]:
from ibm_watson_machine_learning import APIClient

In [ ]:
pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()
WML_API_KEY = os.getenv('WML-API-KEY')
# add the WMl-API-KEY to .evn file to get picked up here. Ensure .env file is added to .gitignore so you don't commit it to GitHub.
wml_credentials = {
                   "url": "https://eu-gb.ml.cloud.ibm.com",
                   "apikey":WML_API_KEY
                  }

client = APIClient(wml_credentials)

In [ ]:
space_metadata = {
    'name': 'taketwo-wml-space',
    'description': 'taketwo-wml-space',
}

In [ ]:
#obtained this from WML resource on IBM Cloud
space_id = '2d3e8ea7-efa2-4efd-ae26-8a46e7a714df'

In [ ]:
client.spaces.list(limit=10)

In [ ]:
client.set.default_space(space_id)

In [ ]:
sofware_spec_uid = client.software_specifications.get_id_by_name("default_py3.7")

In [ ]:
trained_model_SGD

In [ ]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'Take Two Scikit model',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

# commenting out for now
published_model = client.repository.store_model(
    model=trained_model_SGD.text_clf,
    meta_props=metadata,
    training_data=None,
    training_target=None)


In [ ]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

In [ ]:
models_details = client.repository.list_models()

In [ ]:
client.deployments.list()
# use the first GUID in the scoring part

In [ ]:
published_model_uid

In [ ]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "take-two-deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)

In [ ]:
client.deployments.list()

In [ ]:
deployment_uid = client.deployments.get_uid(created_deployment)
deployment_uid

In [ ]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

In [ ]:
client.deployments.get_details(deployment_uid)


In [ ]:
marker_test.data[0]

In [ ]:
# 1. count_vect
# 2. tf_transformer
# 3. clf

pipeline = trained_model_SGD.text_clf.steps

In [ ]:
# client.repository.download()

In [ ]:
vect = pipeline[0][1]
transformer = pipeline[1][1]
clf = pipeline[2][1]

In [ ]:
type(clf)

In [ ]:
# trying to create input format that is needed for WML
count_vect = trained_model_NB.count_vect
tf_transformer = trained_model_NB.tf_transformer
clf = trained_model_NB.clf
X_new_counts = count_vect.transform(marker_test.data[1:10])
X_new_tfidf = tf_transformer.transform(X_new_counts)


valuesList = X_new_tfidf.todense().tolist()
payload_scoring = {"input_data": [{"values": valuesList}]}



# def model_test(flagger_new_test, trained_model):
#     count_vect = trained_model.count_vect
#     tf_transformer = trained_model.tf_transformer
#     clf = trained_model.clf
#     X_new_counts = count_vect.transform(flagger_new_test)
#     X_new_tfidf = tf_transformer.transform(X_new_counts)
#     predicted = clf.predict(X_new_tfidf)
#     return predicted

In [ ]:
print(deployment_uid)
predictions = client.deployments.score(deployment_uid, payload_scoring)

In [ ]:
predictions

In [ ]:
# prediction with WML

import requests

# NOTE: you must manually set API_KEY below using information retrieved from your IBM Cloud account.
API_KEY = wml_credentials['apikey']
token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey": API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
# payload_scoring = {"input_data": [{"values": [X_new_tfidf.toarray()]}]}

response_scoring = requests.post('https://eu-gb.ml.cloud.ibm.com/ml/v4/deployments/11633376-4d22-4934-a7ec-6e5cb3a23293/predictions?version=2021-07-16', json=payload_scoring, headers={'Authorization': 'Bearer ' + mltoken})
print("Scoring response")
print(response_scoring.json())